# Dependencies

Feel free to modify the seed to show different images!  
Just make sure different models are compared on the same set of images.

In [13]:
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
import torchvision.transforms as standard_transforms
from torch.utils.data import DataLoader
from voc import VOC

import numpy as np

torch.manual_seed(2025)

# Mapping Class to Color

In [14]:
'''
color map
0=background, 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle # 6=bus, 7=car, 8=cat, 9=chair, 10=cow, 11=diningtable,
12=dog, 13=horse, 14=motorbike, 15=person # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
'''

noToClass = {
    0:"background", 1:"aeroplane", 2:"bicycle", 3:"bird", 4:"boat", 5:"bottle", 6:"bus", 7:"car", 8:"cat", 9:"chair", 10:"cow", 11:"dining table",
12:"dog", 13:"horse", 14:"motorbike", 15:"person", 16:"potted plant", 17:"sheep", 18:"sofa", 19:"train", 20:"tv/monitor"
}

palette = [0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 128, 0, 0, 0, 128, 128, 0, 128, 0, 128, 128,
           128, 128, 128, 64, 0, 0, 192, 0, 0, 64, 128, 0, 192, 128, 0, 64, 0, 128, 192, 0, 128,
           64, 128, 128, 192, 128, 128, 0, 64, 0, 128, 64, 0, 0, 192, 0, 128, 192, 0, 0, 64, 128]  #3 values- R,G,B for every class. First 3 values for class 0, next 3 for

"""
    Convert palette to map class number to RGB color.

    Expected output, python dictionary with 21 integer keys, each value being a array of size 3
"""
dicti = {}
c = 0
for i in range(0,len(palette),3):
    dicti[c] = palette[i:i+3]
    c+=1

palette = dicti

# DataLoading

In [15]:
# TODO Rewrite the following transformations if needed for your models.
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
mask_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [16]:
batch_size = 1
DS = VOC("test", img_transform, mask_transform)
DL = DataLoader(DS, batch_size=batch_size, shuffle=True)

first_batch = iter(DL)

RuntimeError: Found 0 images, please check the data set

In [ ]:
imgs, msks = [], []
for _ in range(5):
    img, msk = next(first_batch)
    imgs.append(img)
    msks.append(msk)

# Show Image

In [ ]:
def makeImagesPlotReady(msk, channels = 3):
    """
    Similar to how you converted the images, make the mask tensor matplotlib ready

    args:
        msk or img - input mask tensor of shape [1, 3, 224, 224] or numpy of shape (3, 224, 224)
    returns:
        mask/img - mask that is ready to be plotted using matplotlib.
                    Hint: Numpy class of shape (224, 224, 3)
    """

    return (np.rot90(np.rot90(np.rot90((msk.reshape(channels,224,224)).T))))

In [ ]:
output_images = [makeImagesPlotReady(img, 3) for img in imgs]

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(12, 6))
axs = axs.flatten()
for i in range(len(output_images)):
    axs[i].imshow(output_images[i])
    axs[i].axis('off')
plt.show()

# Show Ground Truth Mask

In [ ]:
def convertMaskToRGB(msk):
    """
    Given a msk, convert it into rgb for visulization.
    """
    msk = (msk*255).int()
    msk[msk==255]=0
    msk = msk.reshape((224, 224))
    
    mask_rgb = np.zeros((3, 224, 224), dtype=np.uint8) # Empty array to store RGB of each class
        
    for idx, color in palette.items(): # Converting to colors
        for j in range(3): # R,G,B            
            mask_rgb[j][msk == idx] = color[j]
            
    return mask_rgb

In [ ]:
output_msks = [makeImagesPlotReady(convertMaskToRGB(msk), 3) for msk in msks]

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(12, 6))
axs = axs.flatten()
for i in range(len(output_msks)):
    axs[i].imshow(output_msks[i])
    axs[i].axis('off')
plt.show()

# Plot Image with the mask

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(12, 6))
axs = axs.flatten()
for i in range(len(output_msks)):
    axs[i].imshow(output_images[i])
    axs[i].imshow(output_msks[i], alpha = 0.7)
    axs[i].axis('off')
plt.show()

# TODO: Visualize Your Model's Predicted Mask

1. Load your model
2. Use the model to predict those images to obtain masks (hint: use your exportModel function)
3. Visualize those masks (without and with images) like we did above